# Tool Ahead Of Time Tutorial

Lets jump straight into this tutorial, as time waits for no one 😊

This tutorial uses the DeepSeek-R1 model, but this tutorial can also be applied to other models available through Langchain's ChatOpenAI library.

First copy the source code in the 'src' folder in this repo and run the code, as below:

In [1]:
from typing import List, Callable
from pydantic import BaseModel, Field, TypeAdapter
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.runnables import Runnable
import re

class ToolCall(BaseModel):
    tool: str = Field(..., description="Name of the tool to call")
    args: dict = Field(..., description="Arguments to pass to the tool")

class ManualToolAgent(Runnable):
    """
    A custom agent that handles tools manually.
    """
    def __init__(self, model: ChatOpenAI, tools: List[Callable]):
        self.model = model
        self.tools = tools
        self.json_parser = JsonOutputParser(pydantic_object=ToolCall)
        self.base_executor = create_react_agent(model, tools=[])
    
    def convert_messages(self, messages: List[dict]) -> List[SystemMessage | HumanMessage | AIMessage]:
        """
        Convert dictionary-based messages to LangChain message objects.
        """
        converted_messages = []
        
        message_types = {
            "system": SystemMessage,
            "user": HumanMessage,
            "assistant": AIMessage
        }
        
        for message in messages:
            role = message["role"]
            content = message["content"]
            
            if role in message_types:
                MessageClass = message_types[role]
                converted_message = MessageClass(content=content)
                converted_messages.append(converted_message)
                
        return converted_messages
    
    def invoke(self, inputs: dict) -> dict:
        """
        Execute the agent with manual tool handling.
        
        Args:
            inputs (dict): Dictionary containing messages
            
        Returns:
            dict: Response containing processed message
        """
        # Get messages
        messages = inputs["messages"]
        system_msg = messages[0]
        previous_message = messages[1] if len(messages) > 2 else []
        human_msg = messages[-1]
        
        # Create messages with proper format
        formatted_system_msg = SystemMessage(
            content=(
                "You are a helpful assistant with access to specific tools. "
                "When a user's question matches a tool's capability, you MUST use that tool. "
                "Do not try to solve problems manually if a tool exists for that purpose.\n\n"
                f"{system_msg}\n"
                "Output ONLY a JSON object (with no extra text) that adheres EXACTLY to the following schema:\n\n"
                f"{self.json_parser.get_format_instructions()}\n\n"
                "If the user's question doesn't require any tool, answer directly in plain text with no JSON."
            )
        )
        
        # Add previous messages
        formatted_messages = [{"role": "system", "content": formatted_system_msg.content}]
        if previous_message:
            if isinstance(previous_message, list):
                formatted_messages.extend(previous_message)
        
        # Convert messages to LangChain format
        converted_formatted_messages = self.convert_messages(formatted_messages)
        
        # Add human message
        formatted_human_msg = HumanMessage(content=human_msg)
        all_messages = converted_formatted_messages + [formatted_human_msg]
        
        # Get response from base executor
        response = self.base_executor.invoke({"messages": all_messages})
        last_response = response["messages"][-1].content
        
        # Process JSON response
        matches = re.findall(r'(\{.*?\})', last_response, re.DOTALL)
        json_text = None
        for m in matches:
            if '"tool"' in m and '"args"' in m:
                json_text = m
                break
        
        if json_text:
            try:
                adapter = TypeAdapter(ToolCall)
                parsed = self.json_parser.parse(json_text)
                
                if isinstance(parsed, dict):
                    tool_call = adapter.validate_python(parsed)
                else:
                    tool_call = parsed
                
                # Find the matching tool
                tool_dict = {tool.name: tool for tool in self.tools}
                
                if tool_call.tool in tool_dict:
                    result = tool_dict[tool_call.tool].invoke(tool_call.args)
                else:
                    result = "Error: Unknown tool"
            except Exception as e:
                result = f"Error processing tool call: {str(e)}"
        else:
            result = re.sub(r'<think>.*?</think>', '', last_response, flags=re.DOTALL).strip()
        
        return {"messages": [{"content": result}]}

def create_react_agent_manual_tool(model: ChatOpenAI, tools: List[Callable]) -> ManualToolAgent:
    """
    Create a React agent with manual tool handling.
    
    Args:
        model (ChatOpenAI): The language model to use
        tools (List[Callable]): List of tool functions
        
    Returns:
        ManualToolAgent: Agent with manual tool handling
    """
    return ManualToolAgent(model, tools)

# Creating Tools

Next, create a tool function using LangChain's @tool decorator.

This is just any function (with inputs and outputs) and @tool added at the top of the function.

I have created two tool functions 'calculator' and 'text_analyzer' below:

In [2]:
from langchain_core.tools import tool

@tool
def calculator(expression: str) -> str:
    """Evaluate a math expression."""
    try:
        expression = expression.strip()
        if not expression:
            return "Error: Empty expression"
        
        allowed_chars = set("0123456789+-*/(). ")
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters in expression"
            
        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def text_analyzer(text: str, analysis_type: str) -> str:
    """
    Analyze text to count either words or characters.
    
    Args:
        text (str): The text to analyze
        analysis_type (str): Either 'words' or 'chars'
    """
    try:
        text = text.strip()
        if not text:
            return "Error: Empty text"
            
        if analysis_type.lower() == 'words':
            word_count = len(text.split())
            return f"{word_count}"
        elif analysis_type.lower() == 'chars':
            char_count = len(text)
            return f"{char_count}"
        else:
            return "Error: analysis_type must be either 'words' or 'chars'"
    except Exception as e:
        return f"Error: {str(e)}"

# Initialize Model

Now, initialize a model instance using the format below. 

In this tutorial, I am using the DeepSeek-R1 model hosted on the platform OpenRouter. This model hosted on OpenRouter is available on Langchain's ChatOpenAI library.

If you want to use another model, you will need to check if your model (hosted on whichever platform you have chosen, for eg. Azure, Together AI or DeepSeek's own platform etc.) is first available on Langchain's ChatOpenAI library, and then change the values of the parameters `model`, `api_key` and `base_url` below according to which model and platform you have chosen.

Also note the model you have chosen will determine how the response output of the model will look like. For eg., in the source code above, `re.sub(r'<think>.*?</think>', '', last_response, flags=re.DOTALL).strip()` indicates how the response output for the DeepSeek-R1 model looks like, so this line of code in the source code above will need to be changed according to how the response output for the model you have chosen will look like. This will be the only line of code in the source code above you will need to customize for the model you have chosen.

In [3]:
import os
from dotenv import load_dotenv

# Load environment variable (ie. API key) from .env file
load_dotenv()

# Initialize model
model = ChatOpenAI(
    model="deepseek/deepseek-r1",
    api_key=os.environ["OPENROUTER_API_KEY"],
    base_url="https://openrouter.ai/api/v1"
)

# Previous Messages

Next, if you already have a history of previous messages between the user and the chatbot, store them in the format below.

If you don't have any previous messages, you can use an empty list `previous_messages = []`

Note: The format of the previous messages does not include the system message, the system message has already been included in the source code above (under the variable `formatted_system_msg`). This design is chosen according to current best practices in chatbot design where we isolate the system message from previous messages.

In [4]:
# Example previous messages
previous_messages = [
    # {"role": "system", "content": "You are a helpful AI assistant."}, # Commented out as we do not include system message
    {"role": "user", "content": "What is the capital of Australia?"},
    {"role": "assistant", "content": "The capital of Australia is Canberra."}
]

# Getting Model Response

Finally, now the fun part where we get to see the response of the model using tool calling! 🛠️

To complete the system message in the source code above (under the variable `formatted_system_msg`), the `system_message` variable below needs to STRICTLY follow the format: "When the user's question requires a {tool use}, use the {'corresponding'} tool. For the {'corresponding'} tool, provide the {user message} as a string in the {'user message'} argument in the tool and any {'predefined values'} as a string for all the other arguments in the tool."

For eg. for the 'calculator' tool, since the function for the 'calculator' tool above has one argument 'expression', the `system_message` variable below would be "When the user's question requires a calculation, use the 'calculator' tool. For the 'calculator' tool, provide the user provided math expression as a string in the 'expression' argument in the tool."

For the 'text_analyze' tool, since the function for the 'text_analyze' tool above has two arguments 'text' and 'analysis_type' (where the 'analysis_type' argument has two predefined values 'words' and 'chars'), the `system_message` variable below would be "When the user's question requires analysis the text provided by the user, use the 'text_analyzer' tool. For the 'text_analyzer' tool, provide the user provided text as a string in the 'text' argument in the tool and either 'words' or 'chars' as a string in the 'analysis_type' argument in the tool."

Below are five examples of different combinations of user questions and tools used:

In [5]:
# Example for calculator tool only
system_message = "When the user's question requires a calculation, use the 'calculator' tool. For the 'calculator' tool, provide the user provided math expression as a string in the 'expression' argument in the tool."
user_message = "What is 123 * 456?"
agent_executor_manual_tool = create_react_agent_manual_tool(model, tools=[calculator])
response_manual_tool = agent_executor_manual_tool.invoke({"messages": [system_message, previous_messages, user_message]})
print(response_manual_tool['messages'][0]['content'])

# Example for text analyzer tool only
system_message = "When the user's question requires analysis the text provided by the user, use the 'text_analyzer' tool. For the 'text_analyzer' tool, provide the user provided text as a string in the 'text' argument in the tool and either 'words' or 'chars' as a string in the 'analysis_type' argument in the tool."
user_message = "How many words are in this sentence?: I built my 1st Hello World program"
agent_executor_manual_tool = create_react_agent_manual_tool(model, tools=[text_analyzer])
response_manual_tool = agent_executor_manual_tool.invoke({"messages": [system_message, previous_messages, user_message]})
print(response_manual_tool['messages'][0]['content'])

# Example for both tools with user question requiring math calculation
system_message = """When the user's question requires a calculation, use the 'calculator' tool. For the 'calculator' tool, provide the user provided math expression as a string in the 'expression' argument in the tool.
When the user's question requires analysis the text provided by the user, use the 'text_analyzer' tool. For the 'text_analyzer' tool, provide the user provided text as a string in the 'text' argument in the tool and either 'words' or 'chars' as a string in the 'analysis_type' argument in the tool."""
user_message = "What is 123 * 456?"
agent_executor_manual_tool = create_react_agent_manual_tool(model, tools=[calculator, text_analyzer])
response_manual_tool = agent_executor_manual_tool.invoke({"messages": [system_message, previous_messages, user_message]})
print(response_manual_tool['messages'][0]['content'])

# Example for both tools with user question requiring analysis the text
system_message = """When the user's question requires a calculation, use the 'calculator' tool. For the 'calculator' tool, provide the user provided math expression as a string in the 'expression' argument in the tool.
When the user's question requires analysis the text provided by the user, use the 'text_analyzer' tool. For the 'text_analyzer' tool, provide the user provided text as a string in the 'text' argument in the tool and either 'words' or 'chars' as a string in the 'analysis_type' argument in the tool."""
user_message = "How many words are in this sentence?: I built my 1st Hello World program"
agent_executor_manual_tool = create_react_agent_manual_tool(model, tools=[calculator, text_analyzer])
response_manual_tool = agent_executor_manual_tool.invoke({"messages": [system_message, previous_messages, user_message]})
print(response_manual_tool['messages'][0]['content'])

# Example for both tools with user question not requiring any tools
system_message = """When the user's question requires a calculation, use the 'calculator' tool. For the 'calculator' tool, provide the user provided math expression as a string in the 'expression' argument in the tool.
When the user's question requires analysis the text provided by the user, use the 'text_analyzer' tool. For the 'text_analyzer' tool, provide the user provided text as a string in the 'text' argument in the tool and either 'words' or 'chars' as a string in the 'analysis_type' argument in the tool."""
user_message = "How many languages are there in the world?"
agent_executor_manual_tool = create_react_agent_manual_tool(model, tools=[calculator, text_analyzer])
response_manual_tool = agent_executor_manual_tool.invoke({"messages": [system_message, previous_messages, user_message]})
print(response_manual_tool['messages'][0]['content'])

56088
7
56088
7
The exact number of languages in the world is difficult to determine, but estimates suggest there are between 6,000 to 7,000 languages spoken globally. However, many of these are endangered and at risk of disappearing.


# Summary

Putting all the scripts above together:

In [7]:
from typing import List, Callable
from pydantic import BaseModel, Field, TypeAdapter
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.runnables import Runnable
import re
from langchain_core.tools import tool
import os
from dotenv import load_dotenv

class ToolCall(BaseModel):
    tool: str = Field(..., description="Name of the tool to call")
    args: dict = Field(..., description="Arguments to pass to the tool")

class ManualToolAgent(Runnable):
    """
    A custom agent that handles tools manually.
    """
    def __init__(self, model: ChatOpenAI, tools: List[Callable]):
        self.model = model
        self.tools = tools
        self.json_parser = JsonOutputParser(pydantic_object=ToolCall)
        self.base_executor = create_react_agent(model, tools=[])
    
    def convert_messages(self, messages: List[dict]) -> List[SystemMessage | HumanMessage | AIMessage]:
        """
        Convert dictionary-based messages to LangChain message objects.
        """
        converted_messages = []
        
        message_types = {
            "system": SystemMessage,
            "user": HumanMessage,
            "assistant": AIMessage
        }
        
        for message in messages:
            role = message["role"]
            content = message["content"]
            
            if role in message_types:
                MessageClass = message_types[role]
                converted_message = MessageClass(content=content)
                converted_messages.append(converted_message)
                
        return converted_messages
    
    def invoke(self, inputs: dict) -> dict:
        """
        Execute the agent with manual tool handling.
        
        Args:
            inputs (dict): Dictionary containing messages
            
        Returns:
            dict: Response containing processed message
        """
        # Get messages
        messages = inputs["messages"]
        system_msg = messages[0]
        previous_message = messages[1] if len(messages) > 2 else []
        human_msg = messages[-1]
        
        # Create messages with proper format
        formatted_system_msg = SystemMessage(
            content=(
                "You are a helpful assistant with access to specific tools. "
                "When a user's question matches a tool's capability, you MUST use that tool. "
                "Do not try to solve problems manually if a tool exists for that purpose.\n\n"
                f"{system_msg}\n"
                "Output ONLY a JSON object (with no extra text) that adheres EXACTLY to the following schema:\n\n"
                f"{self.json_parser.get_format_instructions()}\n\n"
                "If the user's question doesn't require any tool, answer directly in plain text with no JSON."
            )
        )
        
        # Add previous messages
        formatted_messages = [{"role": "system", "content": formatted_system_msg.content}]
        if previous_message:
            if isinstance(previous_message, list):
                formatted_messages.extend(previous_message)
        
        # Convert messages to LangChain format
        converted_formatted_messages = self.convert_messages(formatted_messages)
        
        # Add human message
        formatted_human_msg = HumanMessage(content=human_msg)
        all_messages = converted_formatted_messages + [formatted_human_msg]
        
        # Get response from base executor
        response = self.base_executor.invoke({"messages": all_messages})
        last_response = response["messages"][-1].content
        
        # Process JSON response
        matches = re.findall(r'(\{.*?\})', last_response, re.DOTALL)
        json_text = None
        for m in matches:
            if '"tool"' in m and '"args"' in m:
                json_text = m
                break
        
        if json_text:
            try:
                adapter = TypeAdapter(ToolCall)
                parsed = self.json_parser.parse(json_text)
                
                if isinstance(parsed, dict):
                    tool_call = adapter.validate_python(parsed)
                else:
                    tool_call = parsed
                
                # Find the matching tool
                tool_dict = {tool.name: tool for tool in self.tools}
                
                if tool_call.tool in tool_dict:
                    result = tool_dict[tool_call.tool].invoke(tool_call.args)
                else:
                    result = "Error: Unknown tool"
            except Exception as e:
                result = f"Error processing tool call: {str(e)}"
        else:
            result = re.sub(r'<think>.*?</think>', '', last_response, flags=re.DOTALL).strip()
        
        return {"messages": [{"content": result}]}

def create_react_agent_manual_tool(model: ChatOpenAI, tools: List[Callable]) -> ManualToolAgent:
    """
    Create a React agent with manual tool handling.
    
    Args:
        model (ChatOpenAI): The language model to use
        tools (List[Callable]): List of tool functions
        
    Returns:
        ManualToolAgent: Agent with manual tool handling
    """
    return ManualToolAgent(model, tools)

@tool
def calculator(expression: str) -> str:
    """Evaluate a math expression."""
    try:
        expression = expression.strip()
        if not expression:
            return "Error: Empty expression"
        
        allowed_chars = set("0123456789+-*/(). ")
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters in expression"
            
        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def text_analyzer(text: str, analysis_type: str) -> str:
    """
    Analyze text to count either words or characters.
    
    Args:
        text (str): The text to analyze
        analysis_type (str): Either 'words' or 'chars'
    """
    try:
        text = text.strip()
        if not text:
            return "Error: Empty text"
            
        if analysis_type.lower() == 'words':
            word_count = len(text.split())
            return f"{word_count}"
        elif analysis_type.lower() == 'chars':
            char_count = len(text)
            return f"{char_count}"
        else:
            return "Error: analysis_type must be either 'words' or 'chars'"
    except Exception as e:
        return f"Error: {str(e)}"
    
# Load environment variable (ie. API key) from .env file
load_dotenv()

# Initialize model
model = ChatOpenAI(
    model="deepseek/deepseek-r1",
    api_key=os.environ["OPENROUTER_API_KEY"],
    base_url="https://openrouter.ai/api/v1"
)

# Example previous messages
previous_messages = [
    # {"role": "system", "content": "You are a helpful AI assistant."}, # Commented out as we do not include system message
    {"role": "user", "content": "What is the capital of Australia?"},
    {"role": "assistant", "content": "The capital of Australia is Canberra."}
]

# Example for calculator tool only
system_message = "When the user's question requires a calculation, use the 'calculator' tool. For the 'calculator' tool, provide the user provided math expression as a string in the 'expression' argument in the tool."
user_message = "What is 123 * 456?"
agent_executor_manual_tool = create_react_agent_manual_tool(model, tools=[calculator])
response_manual_tool = agent_executor_manual_tool.invoke({"messages": [system_message, previous_messages, user_message]})
print(response_manual_tool['messages'][0]['content'])

# Example for text analyzer tool only
system_message = "When the user's question requires analysis the text provided by the user, use the 'text_analyzer' tool. For the 'text_analyzer' tool, provide the user provided text as a string in the 'text' argument in the tool and either 'words' or 'chars' as a string in the 'analysis_type' argument in the tool."
user_message = "How many words are in this sentence?: I built my 1st Hello World program"
agent_executor_manual_tool = create_react_agent_manual_tool(model, tools=[text_analyzer])
response_manual_tool = agent_executor_manual_tool.invoke({"messages": [system_message, previous_messages, user_message]})
print(response_manual_tool['messages'][0]['content'])

# Example for both tools with user question requiring math calculation
system_message = """When the user's question requires a calculation, use the 'calculator' tool. For the 'calculator' tool, provide the user provided math expression as a string in the 'expression' argument in the tool.
When the user's question requires analysis the text provided by the user, use the 'text_analyzer' tool. For the 'text_analyzer' tool, provide the user provided text as a string in the 'text' argument in the tool and either 'words' or 'chars' as a string in the 'analysis_type' argument in the tool."""
user_message = "What is 123 * 456?"
agent_executor_manual_tool = create_react_agent_manual_tool(model, tools=[calculator, text_analyzer])
response_manual_tool = agent_executor_manual_tool.invoke({"messages": [system_message, previous_messages, user_message]})
print(response_manual_tool['messages'][0]['content'])

# Example for both tools with user question requiring analysis the text
system_message = """When the user's question requires a calculation, use the 'calculator' tool. For the 'calculator' tool, provide the user provided math expression as a string in the 'expression' argument in the tool.
When the user's question requires analysis the text provided by the user, use the 'text_analyzer' tool. For the 'text_analyzer' tool, provide the user provided text as a string in the 'text' argument in the tool and either 'words' or 'chars' as a string in the 'analysis_type' argument in the tool."""
user_message = "How many words are in this sentence?: I built my 1st Hello World program"
agent_executor_manual_tool = create_react_agent_manual_tool(model, tools=[calculator, text_analyzer])
response_manual_tool = agent_executor_manual_tool.invoke({"messages": [system_message, previous_messages, user_message]})
print(response_manual_tool['messages'][0]['content'])

# Example for both tools with user question not requiring any tools
system_message = """When the user's question requires a calculation, use the 'calculator' tool. For the 'calculator' tool, provide the user provided math expression as a string in the 'expression' argument in the tool.
When the user's question requires analysis the text provided by the user, use the 'text_analyzer' tool. For the 'text_analyzer' tool, provide the user provided text as a string in the 'text' argument in the tool and either 'words' or 'chars' as a string in the 'analysis_type' argument in the tool."""
user_message = "How many languages are there in the world?"
agent_executor_manual_tool = create_react_agent_manual_tool(model, tools=[calculator, text_analyzer])
response_manual_tool = agent_executor_manual_tool.invoke({"messages": [system_message, previous_messages, user_message]})
print(response_manual_tool['messages'][0]['content'])

56088
7
56088
7
The exact number of languages in the world is difficult to pin down, but estimates suggest there are approximately **7,000 languages** spoken globally. This number fluctuates due to factors like language extinction, emergence of new dialects, and varying definitions of what constitutes a distinct language. Major sources like *Ethnologue* provide this commonly cited figure.
